In [11]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

In [12]:
# Set up Selenium options and driver
options = Options()
options.add_argument("--disable-gpu")
# options.add_argument("--headless")  # Optional: Run in headless mode

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Navigate to the page
base_url = "https://www.nykaa.com/brands/nivea/c/645?"
params = "ptype=lst&id=645&root=brand_menu,brand_list,NIVEA"
paginationNumber = 1

# Allow initial time for the page to load
time.sleep(5)

# Scroll incrementally until all products are loaded
scroll_pause_time = 2  # Wait time after each scroll

# Lists to store data
productLinks = []
products = []

while True:

    if paginationNumber == 1:
        url = f"{base_url}{params}"
    else:
        url = f"{base_url}page_no={paginationNumber}&sort=popularity&ptype=lst&id=645&root=brand_menu,brand_list,NIVEA&eq=desktop"
    
    # Load the Page
    driver.get(url)
    print(f"Scraping Page {paginationNumber}: {url}")
    time.sleep(3)
    
    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

    # Wait until the products are loaded
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.productWrapper.css-17nge1h'))
        )
    except TimeoutException:
        print(f"Scraped the Total number of pages: {paginationNumber - 1}")
        break

    # Parse the page content with BeautifulSoup
    content = driver.page_source
    soup = BeautifulSoup(content, 'html.parser')
    
    # Find all product items
    deygaOrganicProducts = soup.findAll('div', class_='productWrapper css-17nge1h')
    if not deygaOrganicProducts:
        print("No Products found on this page.")
        break

    # Extract product links
    for product in deygaOrganicProducts:
        # Extract the Stock Status
        stockTag = product.find('span', class_ = "css-lg5xc9")
        if stockTag:
            stock = stockTag.get_text(strip = True)
        else:
            stock = "In Stock"

        # Extract the Product Highlight
        highlightTag = product.find("li", class_ = "custom-tag css-1bse542")
        if highlightTag:
            highlight = highlightTag.get_text(strip = True)
        else:
            highlight = "None"

        # Extract product link
        linkTag = product.find('a', href=True)
        if linkTag:
            productLink = linkTag['href']
            # Check if the link is a relative URL and add the base URL if necessary
            if not productLink.startswith("http"):  # Relative URL
                productLink = "https://www.nykaa.com" + productLink
            productLinks.append({"link" : productLink, "stock" : stock, "highlight" : highlight})
                
    for productData in productLinks:
        
        link = productData["link"]
        stock = productData["stock"]
        highlight = productData["highlight"]
        
        driver.get(link)
        time.sleep(3)  # Allow time for the page to load

        # Parse individual product page
        productPageContent = driver.page_source
        productSoup = BeautifulSoup(productPageContent, 'html.parser')
    
        # Extract Product Name
        productTag = productSoup.find('h1', class_='css-1gc4x7i')
        if productTag:
            prodName = productTag.get_text(strip=True)
        else:
            prodName = "No Description Available"
    
        # Extract Ratings
        ratingTag = productSoup.find('div', class_="css-1m0y15j")
        if ratingTag:
            ratingText = ratingTag.find('div', class_="css-m6n3ou")
            if ratingText:
                ratings = ratingText.get_text(strip = True)
            else:
                ratings = "N/A"
        else:
            ratings = "N/A"

        # Extract the Ratings & Reviews Count
        ratingCountTag = productSoup.find('div', class_ = "css-1eip5u4")
        if ratingCountTag:
            ratingCount = ratingCountTag.get_text(strip = True)
        else:
            ratingCount = "None"

        # Extract the Offer Price and Original Price
        productDiv = productSoup.find('div', class_ = "css-1d0jf8e")

        # Extract the first two (span) tags
        if productDiv:
            spanTags = productDiv.find_all('span')
            # print(len(spanTags))

            originalPrice = "N/A"
            offerPrice = "N/A"

            if len(spanTags) == 2:
                originalPrice = spanTags[1].get_text(strip = True)
                offerPrice = "N/A"
            elif len(spanTags) == 4:
                originalPrice = spanTags[1].get_text(strip = True)
                offerPrice = spanTags[2].get_text(strip = True)

            # Print the Values
            # print(f"Original Price: {originalPrice}, Offer Price: {offerPrice}")

        else:
            print("Product div not found, defaulting to Unavailable for prices")

        # Extract the Discount
        discountTag = productSoup.find('span', class_ = "css-bhhehx")
        if discountTag:
            discount = discountTag.get_text(strip = True)
        else:
            discount = "No Discount"

        products.append({
            "Product Brand" : "Nivea",
            "Product Name" : prodName,
            "Product Rating" : ratings,
            "Product Rating & Review Count" : ratingCount,
            "Product Original Price" : originalPrice, 
            "Product Offer Price" : offerPrice, 
            "Product Discount" : discount,
            "Product Highlight" : highlight,
            "Product Stock Status" : stock
        })

    # Clear the product links list to avoid the duplication
    productLinks.clear()

    paginationNumber += 1 # Increment the Page Number
    time.sleep(5)

# Close the browser after scraping
driver.quit()

Scraping Page 1: https://www.nykaa.com/brands/nivea/c/645?ptype=lst&id=645&root=brand_menu,brand_list,NIVEA
Scraping Page 2: https://www.nykaa.com/brands/nivea/c/645?page_no=2&sort=popularity&ptype=lst&id=645&root=brand_menu,brand_list,NIVEA&eq=desktop
Scraping Page 3: https://www.nykaa.com/brands/nivea/c/645?page_no=3&sort=popularity&ptype=lst&id=645&root=brand_menu,brand_list,NIVEA&eq=desktop
Scraping Page 4: https://www.nykaa.com/brands/nivea/c/645?page_no=4&sort=popularity&ptype=lst&id=645&root=brand_menu,brand_list,NIVEA&eq=desktop
Scraping Page 5: https://www.nykaa.com/brands/nivea/c/645?page_no=5&sort=popularity&ptype=lst&id=645&root=brand_menu,brand_list,NIVEA&eq=desktop
Scraping Page 6: https://www.nykaa.com/brands/nivea/c/645?page_no=6&sort=popularity&ptype=lst&id=645&root=brand_menu,brand_list,NIVEA&eq=desktop
Scraping Page 7: https://www.nykaa.com/brands/nivea/c/645?page_no=7&sort=popularity&ptype=lst&id=645&root=brand_menu,brand_list,NIVEA&eq=desktop
Scraping Page 8: https

In [13]:
print(f"Products: \n {products}")

Products: 
 [{'Product Brand': 'Nivea', 'Product Name': 'NIVEA Luminous Even Glow Day and Night Cream Combo(2 pcs)', 'Product Rating': '4.3/5', 'Product Rating & Review Count': '12ratings&6reviews', 'Product Original Price': '₹948', 'Product Offer Price': '₹758', 'Product Discount': '20% Off', 'Product Highlight': 'None', 'Product Stock Status': 'In Stock'}, {'Product Brand': 'Nivea', 'Product Name': 'NIVEA Vit E Body Milk Lotion - 5 In 1 Complete Care For 48H Nourished & Smooth Skin (Very Dry Skin)(600ml)', 'Product Rating': '4.5/5', 'Product Rating & Review Count': '62582ratings&4175reviews', 'Product Original Price': '₹724', 'Product Offer Price': '₹615', 'Product Discount': '15% Off', 'Product Highlight': 'BESTSELLER', 'Product Stock Status': 'In Stock'}, {'Product Brand': 'Nivea', 'Product Name': 'NIVEA Soft Vit E Non-Sticky Face Moisturizer- Fresh, Soft & Hydrated Skin(200ml)', 'Product Rating': '4.5/5', 'Product Rating & Review Count': '52961ratings&3885reviews', 'Product Origin

In [14]:
nivea  = pd.DataFrame(products)
nivea

,Product Brand,Product Name,Product Rating,Product Rating & Review Count,Product Original Price,Product Offer Price,Product Discount,Product Highlight,Product Stock Status
0,Nivea,NIVEA Luminous Even Glow Day and Night Cream C...,4.3/5,12ratings&6reviews,₹948,₹758,20% Off,None,In Stock
1,Nivea,NIVEA Vit E Body Milk Lotion - 5 In 1 Complete...,4.5/5,62582ratings&4175reviews,₹724,₹615,15% Off,BESTSELLER,In Stock
2,Nivea,NIVEA Soft Vit E Non-Sticky Face Moisturizer- ...,4.5/5,52961ratings&3885reviews,₹400,₹360,10% Off,BESTSELLER,In Stock
3,Nivea,NIVEA Bestselling- Smooth Underarms Deo Roll O...,4.5/5,24125ratings&1198reviews,₹498,₹398,20% Off,None,In Stock
4,Nivea,NIVEA Bestselling Fruity Blackberry Shine + Li...,4.5/5,7657ratings&904reviews,₹559,₹447,20% Off,None,In Stock
...,...,...,...,...,...,...,...,...,...
148,Nivea,NIVEA Milk Delights Moisturizing Honey Face Wa...,4.4/5,15543ratings&1375reviews,₹460,₹437,5% Off,None,Out Of Stock
149,Nivea,NIVEA Fruity Lip Care(10gm),4.2/5,29ratings,₹45,N/A,No Discount,None,Out Of Stock
150,Nivea,"NIVEA Milk & Saffron Face Wash Combo, PH Balan...",4.3/5,10463ratings&917reviews,₹460,₹437,5% Off,None,Out Of Stock
151,Nivea,NIVEA Rose Water Gel Body Lotion Non Sticky Fe...,4.3/5,2499ratings&190reviews,₹149,N/A,No Discount,None,Out Of Stock


In [16]:
# nivea.to_csv("C:/Users/Logiya Vidhyapathy/Documents/KGISL Data Science/Project Files and Documents/Capstone Project/Nivea Products_Nykaa.csv", index = False)